# Assignment 3. Done by Arina Shinkorenok, Group J4132c

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=386e2f5410ab14c94c2302360c97be0c0a4ab454d5751e91cbf9259c0bfa126d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
"""
Create spark session with 2 cores and 2g memory on a driver (you will work locally)
"""

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("Task_3") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

In [ ]:
"""
Using Spark UI link below you can see tasks submitted to spark
Also you can see cached values
"""

In [ ]:
spark

In [ ]:
import random

from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, udf, monotonically_increasing_id, mean

In [ ]:
"""
Read dataset
"""

In [ ]:
data = spark.read.json("cards.json")
data.show(10)

+----------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+--------------------+-------+--------------------+------+---------+---------+---------------+---------+--------------------+---------------+-------------------+--------+--------------------+-----------+----+---------+--------------+-------+-----------+------------------+--------------------+-----------+
|          artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|           entourage|faction|              flavor|health|hideStats|howToEarn|howToEarnGolden|       id|           mechanics|multiClassGroup|               name|overload|    playRequirements|playerClass|race|   rarity|referencedTags|    set|spellDamage|targetingArrowText|                text|       type|
+----------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+--------------------+-------+--------------------+------+---------+---------+---------------+---

In [ ]:
"""
Remove samples without artist name, attack and health value
"""

In [ ]:
data_authors = data.filter(
    (col("artist").isNotNull()) &
    (col("attack").isNotNull()) &
    (col("health").isNotNull())
).cache() # remember about .cache() operation

In [ ]:
"""
For each column print the number of non Null rows and number of unique rows
"""

In [ ]:
for column_name in data_authors.columns:
    rows_number_not_empty = data_authors.filter(col(column_name).isNotNull()).count()
    rows_number_unique = data_authors.select(column_name).distinct().count()

    print(f"{column_name}. Non empty: {rows_number_not_empty}\tUnique values: {rows_number_unique}")

artist. Non empty: 829	Unique values: 234
attack. Non empty: 829	Unique values: 31
cardClass. Non empty: 829	Unique values: 10
classes. Non empty: 9	Unique values: 4
collectible. Non empty: 731	Unique values: 2
collectionText. Non empty: 5	Unique values: 6
cost. Non empty: 829	Unique values: 14
dbfId. Non empty: 829	Unique values: 829
durability. Non empty: 0	Unique values: 1
elite. Non empty: 141	Unique values: 2
entourage. Non empty: 6	Unique values: 7
faction. Non empty: 60	Unique values: 3
flavor. Non empty: 731	Unique values: 732
health. Non empty: 829	Unique values: 30
hideStats. Non empty: 2	Unique values: 2
howToEarn. Non empty: 136	Unique values: 27
howToEarnGolden. Non empty: 181	Unique values: 77
id. Non empty: 829	Unique values: 829
mechanics. Non empty: 553	Unique values: 49
multiClassGroup. Non empty: 9	Unique values: 4
name. Non empty: 829	Unique values: 791
overload. Non empty: 9	Unique values: 4
playRequirements. Non empty: 81	Unique values: 34
playerClass. Non empty: 

In [ ]:
data_authors.select("race").distinct().show()

+----------+
|      race|
+----------+
|MECHANICAL|
|    MURLOC|
|    DRAGON|
|     TOTEM|
|     BEAST|
|     DEMON|
|    PIRATE|
|      NULL|
+----------+



In [ ]:
data_authors.select("rarity").distinct().show()

+---------+
|   rarity|
+---------+
|     FREE|
|     EPIC|
|     RARE|
|   COMMON|
|LEGENDARY|
|     NULL|
+---------+



In [ ]:
str2int_rarity = {
    "FREE": 1,
    "COMMON": 2,
    "RARE": 3,
    "LEGENDARY": 4,
    "EPIC": 5,
    "NULL": 0
}

In [ ]:
"""
Using dict above - create a new column with name "rarity_int", where each string will be
converted to an integer

Operation "col" return "pointer" to required column
"""

In [ ]:
data_authors = data_authors.withColumn("rarity_int", udf(lambda x: str2int_rarity.get(x), IntegerType())(col("rarity")))

In [ ]:
"""
Calculate average rarity of each race
"""

In [ ]:
data_authors.groupby("race").agg(mean("rarity_int").alias("average rarity")).show()

+----------+------------------+
|      race|    average rarity|
+----------+------------------+
|MECHANICAL|3.0185185185185186|
|      NULL| 2.982824427480916|
|    MURLOC|2.9047619047619047|
|    DRAGON|3.3461538461538463|
|     TOTEM|               2.5|
|     BEAST|2.6666666666666665|
|     DEMON|2.8518518518518516|
|    PIRATE|2.8947368421052633|
+----------+------------------+



In [ ]:
@udf(returnType=IntegerType())
def calculate_number_of_attacks(health, attack):
    if attack == 0:
        return -1
    return health // attack + (0 if health % attack == 0 else 1)

In [ ]:
"""
Using udf defined above, calculate how many attacks needed for each card to kill itself
"""

In [ ]:
data_authors_2 = data_authors.withColumn(
    "Attack to kill itself",
    calculate_number_of_attacks(col("health"), col("attack")),
    ).select("health", "attack", "Attack to kill itself")

In [ ]:
data_authors_2.show(5)

+------+------+---------------------+
|health|attack|Attack to kill itself|
+------+------+---------------------+
|    10|    10|                    1|
|     5|     3|                    2|
|     3|     2|                    2|
|     2|     2|                    1|
|     4|     5|                    1|
+------+------+---------------------+
only showing top 5 rows



In [ ]:
"""
Here some work on randomizing indexes for dataframe
It is not always easy to assign unique values for each row non-monotonically
"""

In [ ]:
random_ids_left = list(range(data_authors.count() + 1))
random_ids_right = list(range(data_authors.count() + 1))

In [ ]:
random.shuffle(random_ids_left)
random.shuffle(random_ids_right)

In [ ]:
"""
Create monotonic id
"""

In [ ]:
data_authors = data_authors.withColumn("unique_id", monotonically_increasing_id())

In [ ]:
"""
Create left and right indexes for each sample. You will use them for join operation
"""

In [ ]:
data_authors = data_authors.withColumn("left_id", udf(lambda x: random_ids_left[x], IntegerType())(col("unique_id")))
data_authors = data_authors.withColumn("right_id", udf(lambda x: random_ids_right[x], IntegerType())(col("unique_id"))).cache()

In [ ]:
data_authors.count()

829

In [ ]:
"""
For each sample you need to select another sample, where left_id == right_id and create new row
Resulting dataframe consists of [left_id, name, attack, health, right_id, right_name, right_attack, right_health]
"""

In [ ]:
"""
Small hint: data_authors.select(col('name').alias('right_name'))
"""

In [ ]:
"""
Small hint: you can join table with itself
"""

In [ ]:
data_crossed = data_authors \
    .alias("left") \
    .join(
        data_authors.alias("right"),
        col("left.left_id") == col("right.right_id")
    ) \
    .select(
      col("left.left_id"),
      col("left.name"),
      col("left.attack"),
      col("left.health"),
      col("right.right_id"),
      col("right.name").alias("right_name"),
      col("right.attack").alias("right_attack"),
      col("right.health").alias("right_health"),
)

data_crossed.show(1)

+-------+-------+------+------+--------+---------------+------------+------------+
|left_id|   name|attack|health|right_id|     right_name|right_attack|right_health|
+-------+-------+------+------+--------+---------------+------------+------------+
|    751|Icehowl|    10|    10|     751|Animated Shield|           0|           5|
+-------+-------+------+------+--------+---------------+------------+------------+
only showing top 1 row



In [ ]:
"""
Now calculate how many attacks required for left card to kill right card
"""

In [ ]:
data_crossed = data_crossed.withColumn("Attacks to kill", calculate_number_of_attacks(col("right_health"), col("attack")))

In [ ]:
data_crossed.filter(col("Attacks to kill") > 10).show(100)

+-------+--------------------+------+------+--------+---------------+------------+------------+---------------+
|left_id|                name|attack|health|right_id|     right_name|right_attack|right_health|Attacks to kill|
+-------+--------------------+------+------+--------+---------------+------------+------------+---------------+
|    196|        Echoing Ooze|     1|     2|     196|     Jade Golem|          30|          30|             30|
|    273|         Axe Flinger|     2|     5|     273|     Jade Golem|          29|          29|             15|
|     67|Silverback Patriarch|     1|     4|      67|     Jade Golem|          21|          21|             21|
|    560|       Lance Carrier|     1|     2|     560|     Jade Golem|          28|          28|             28|
|    752| Gnomeregan Infantry|     1|     4|     752|     Jade Golem|          11|          11|             11|
|     14|         Mana Addict|     1|     3|      14|The Ancient One|          30|          30|         

In [ ]:
"""
Additional task OPTIONAL

Calculate who will win (left or right), if left one starts first
"""

### I did the extra task in two ways, because I thought that since the dataset was taken from Hearthstone, it was worth implementing the system from that game (with health reduction for the attacker). But I also implemented the second way, where the attacker's health is not reduced.|

In [ ]:
result_and_turns_struct = StructType([
    StructField(name="result", dataType=StringType(), nullable=False),
    StructField(name="turns", dataType=IntegerType(), nullable=False),
])

### 1. Hearthstone system
In Hearthstone, where the dataset came from, there is a system where the attacking card also takes damage equal to the attack score of the attacked card. So in this case it makes no difference who starts attacking first. Also with this system there may not be a winner, both cards can be destroyed when one of them is hit. For such a case we use the notation "draw".

In [ ]:
def get_winner_and_turns_hearthstone(left_health, left_attack, right_health, right_attack):
    turns = 0
    while left_health > 0 and right_health > 0:
        right_health -= left_attack
        left_health -= right_attack
        turns += 1

        if right_health <= 0 and left_health > 0:
            return ("left", turns)
        if left_health <= 0 and right_health > 0:
            return ("right", turns)
    return ("draw", turns)

udf_winner_and_turns_hearthstone = udf(get_winner_and_turns_hearthstone, result_and_turns_struct)

In [ ]:
# Removing the "Attacks to kill" column, because it's the wrong logic in
# determining the winner, because cards on the right side can win too
data_with_fight_results_hearthstone = data_crossed.drop("Attacks to kill")

# Writing a new column with type Struct, which contains information about the
# result of the battle and the number of turns
data_with_fight_results_hearthstone = data_with_fight_results_hearthstone.withColumn(
    "result_and_turns",
    udf_winner_and_turns_hearthstone(
        col("health"),
        col("attack"),
        col("right_health"),
        col("right_attack"),
    ),
)

# Splitting this column into two separate columns
data_with_fight_results_hearthstone = data_with_fight_results_hearthstone \
    .withColumn("fight_result", data_with_fight_results_hearthstone["result_and_turns"]["result"]) \
    .withColumn("turns", data_with_fight_results_hearthstone["result_and_turns"]["turns"]) \
    .drop("result_and_turns")

data_with_fight_results_hearthstone.show()

+-------+--------------------+------+------+--------+--------------------+------------+------------+------------+-----+
|left_id|                name|attack|health|right_id|          right_name|right_attack|right_health|fight_result|turns|
+-------+--------------------+------+------+--------+--------------------+------------+------------+------------+-----+
|    751|             Icehowl|    10|    10|     751|     Animated Shield|           0|           5|        left|    1|
|    799|        Wailing Soul|     3|     5|     799|       Flying Monkey|           4|           2|        left|    1|
|     84|   Sunfury Protector|     2|     3|      84|        Baron Geddon|           7|           5|       right|    1|
|    254|        Dark Peddler|     2|     2|     254|      Addled Grizzly|           2|           2|        draw|    1|
|    586|    Genzo, the Shark|     5|     4|     586|     Frostwolf Grunt|           2|           2|        left|    1|
|    294|    Reliquary Seeker|     1|   

In [ ]:
# Rows with number of turns greater than 3
data_with_fight_results_hearthstone.filter(col("turns") > 3).show(100)

+-------+---------------+------+------+--------+-------------------+------------+------------+------------+-----+
|left_id|           name|attack|health|right_id|         right_name|right_attack|right_health|fight_result|turns|
+-------+---------------+------+------+--------+-------------------+------------+------------+------------+-----+
|    553|Public Defender|     0|     7|     553|  Twilight Guardian|           2|           6|       right|    4|
|    479|Novice Engineer|     1|     1|     479|    Summoning Stone|           0|           6|        left|    6|
|    266|     Imp Master|     1|     5|     266|Validated Doomsayer|           0|           7|        left|    7|
|    583|     Lightspawn|     0|     5|     583|       Am'gam Rager|           1|           5|       right|    5|
|     79|       Dorothee|     0|    10|      79|  Brann Bronzebeard|           2|           4|       right|    5|
|    351|     Voidwalker|     1|     3|     351|          Nat Pagle|           0|       

### 2. Default system
This code is written for a case abstracted from Hearthstone, where the attacker does not take damage when attacking. In this case, there can be no draw.

In [ ]:
def get_winner_and_turns_default(left_health, left_attack, right_health, right_attack):
    turns = 0
    while left_health > 0 and right_health > 0:
        # Left attacks
        if turns % 2 == 0:
            right_health -= left_attack
        # Right attacks
        else:
            left_health -= right_attack

        turns += 1

        if right_health <= 0:
            return ("left", turns)
        if left_health <= 0:
            return ("right", turns)

udf_winner_and_turns_default = udf(get_winner_and_turns_default, result_and_turns_struct)

In [ ]:
data_with_fight_results_default = data_crossed.drop("Attacks to kill")

data_with_fight_results_default = data_with_fight_results_default.withColumn(
    "result_and_turns",
    udf_winner_and_turns_default(
        col("health"),
        col("attack"),
        col("right_health"),
        col("right_attack"),
    ),
)

# Splitting this column into two separate columns
data_with_fight_results_default = data_with_fight_results_default \
    .withColumn("winner", data_with_fight_results_default["result_and_turns"]["result"]) \
    .withColumn("turns", data_with_fight_results_default["result_and_turns"]["turns"]) \
    .drop("result_and_turns")

data_with_fight_results_default.show()

+-------+--------------------+------+------+--------+--------------------+------------+------------+------+-----+
|left_id|                name|attack|health|right_id|          right_name|right_attack|right_health|winner|turns|
+-------+--------------------+------+------+--------+--------------------+------------+------------+------+-----+
|    751|             Icehowl|    10|    10|     751|     Animated Shield|           0|           5|  left|    1|
|    799|        Wailing Soul|     3|     5|     799|       Flying Monkey|           4|           2|  left|    1|
|     84|   Sunfury Protector|     2|     3|      84|        Baron Geddon|           7|           5| right|    2|
|    254|        Dark Peddler|     2|     2|     254|      Addled Grizzly|           2|           2|  left|    1|
|    586|    Genzo, the Shark|     5|     4|     586|     Frostwolf Grunt|           2|           2|  left|    1|
|    294|    Reliquary Seeker|     1|     1|     294|        Hozen Healer|           2| 

In [ ]:
# Rows with number of turns greater than 3
data_with_fight_results_default.filter(col("turns") > 3).show(100)

+-------+--------------------+------+------+--------+--------------------+------------+------------+------+-----+
|left_id|                name|attack|health|right_id|          right_name|right_attack|right_health|winner|turns|
+-------+--------------------+------+------+--------+--------------------+------------+------------+------+-----+
|    581|        Shieldbearer|     0|     4|     581|    Argent Protector|           2|           2| right|    4|
|    483| Gurubashi Berserker|     2|     7|     483|      King of Beasts|           2|           6|  left|    5|
|     17|     Mana Tide Totem|     0|     3|      17|Mistress of Mixtures|           2|           2| right|    4|
|    553|     Public Defender|     0|     7|     553|   Twilight Guardian|           2|           6| right|    8|
|    479|     Novice Engineer|     1|     1|     479|     Summoning Stone|           0|           6|  left|   11|
|    530|        Dalaran Mage|     1|     4|     530|     Acolyte of Pain|           1| 